In [2]:
import pandas as pd 
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import json
import mne
import numpy as np
from tqdm import tqdm
import re
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from multiprocessing import Pool
import warnings
from collections import defaultdict
from typing import Dict, Tuple, Any, List

warnings.filterwarnings("ignore")

os.chdir('../../..')

Functions

In [37]:
    def _get_day_time_df(day_time_meta_path):
        # Читаем первый лист
        meta1 = pd.read_excel(day_time_meta_path, sheet_name="1_Сессия", header=1).rename(columns={
            'Subject ID'          : 'Subject_id',
            'Время начала записи' : 'Time',
        })
        meta1['Trial_id'] = 1
        
        # Читаем второй лист
        meta2 = pd.read_excel(day_time_meta_path, sheet_name="2_Сессия", header=1).rename(columns={
            'Subject ID'          : 'Subject_id',
            'Время начала записи' : 'Time',
        })
        meta2['Trial_id'] = 2
        
        # Конкатенация
        meta = pd.concat([meta1, meta2], ignore_index=True)
        
        meta
        
        meta['Subject_id'] = (
            meta['Subject_id']
            .astype(str)
            .str.extract(r'(\d+)', expand=False)
            .astype('float')
        )
        meta = meta[meta['Subject_id'].notna()].copy()
        meta['Subject_id'] = meta['Subject_id'].astype(int)
        
        s  = meta['Time'].astype(str).str.strip()                    
        n  = pd.to_numeric(s, errors='coerce')                        
        dt_str = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
        dt_num = pd.to_datetime(n, errors='coerce', origin='1899-12-30', unit='D')
        dt = dt_str.fillna(dt_num)                                   
        
        meta['Hour'] = dt.dt.hour
        meta['Time'] = dt.dt.strftime('%H:%M')
        
        meta_idx = (meta[['Subject_id', 'Trial_id', 'Hour', 'Time']]
                    .dropna(subset=['Hour'])                         
                    .drop_duplicates(['Subject_id', 'Trial_id'], keep='last'))
        
        meta_idx['Condition'] = pd.cut(
            meta_idx['Hour'],
            bins=[-0.1, 10, 18, 24],
            labels=['Other', 'Day', 'Evening'],
            right=False,
            include_lowest=True
        ).astype(object).fillna('Other')
    
        return meta_idx
    
        
    
    
    def filter_exec_psds(exec_spec_path, 
                         day_time=None,
                         day_time_meta_path=None,
                         stim_type=None, 
                         stim_label=None, 
                         gender=None, 
                         age=None, 
                         handiness=None):
        """
        Filters experimental PSD data (.npz file) based on subject and trial metadata.
    
        :param: exec_spec_path : str - Path to the `.npz` file containing experimental PSD data.
        :param: day_time : str - Daytime condition "Day" or "Evening"
        :param: day_time_meta_path : str - Path to an Excel file with metadata. Required only if `day_time` is provided.
        :param: stim_type : str or list(str) - "g" → geometric or "r" → random.
        :param: stim_label : int or list(int) - Label(s) of stimuli ("-1" → random or "0–12" → geometric).
        :param: gender : str - Gender "m" → male or  "f" → female.
        :param: age : int or list(int) - Age(s).
        :param: handiness : str - Hand preference "r" → right-handed or "l" → left-handed
    
        :return: list - filtered execution spectras [power, phase, subject_id, trial_id, gender, handiness, age, label, img, task_type].
        """
        
        loaded = np.load(exec_spec_path)
        results_arr = []
    
        # Load daytime metadata if filtering by daytime
        if day_time is not None:
            if day_time_meta_path is None:
                raise ValueError("You must provide 'day_time_meta_path' when 'day_time' is specified.")
            day_time_df = _get_day_time_df(day_time_meta_path)
        else:
            day_time_df = None
    
        # Normalize input filters
        if isinstance(stim_label, (list, np.ndarray)):
            stim_label = set(stim_label)
        if isinstance(age, (list, np.ndarray)):
            age = set(age)
        if isinstance(stim_type, str):
            stim_type = {stim_type}
        elif stim_type is not None:
            stim_type = set(stim_type)
    
        # Iterate through all subjects/trials in .npz file
        i = 0
        while f'power_{i}' in loaded:
            power = loaded[f'power_{i}']
            phase = loaded[f'phase_{i}']
            s_id = int(loaded[f'subject_id_{i}'])
            t_id = int(loaded[f'trial_id_{i}'])
            gend = str(loaded[f'gender_{i}'])
            hand = str(loaded[f'handiness_{i}'])
            ag = int(loaded[f'age_{i}'])
            label = int(loaded[f'label_{i}'])
            img = loaded[f'img_{i}']
            task_type = str(loaded[f'task_type_{i}'])
    
            # Apply filters
            # Filter by daytime condition
            if day_time is not None:
                cond_row = day_time_df[
                    (day_time_df["Subject_id"] == s_id) & (day_time_df["Trial_id"] == t_id)
                ]
                if cond_row.empty or cond_row["Condition"].values[0] != day_time:
                    i += 1
                    continue
    
            # Filter by stim_type
            if stim_type is not None and task_type not in stim_type:
                i += 1
                continue
    
            # Filter by stim_label
            if stim_label is not None:
                if isinstance(stim_label, set):
                    if label not in stim_label:
                        i += 1
                        continue
                elif label != stim_label:
                    i += 1
                    continue
    
            # Filter by gender
            if gender is not None and gend != gender:
                i += 1
                continue
    
            # Filter by age
            if age is not None:
                if isinstance(age, set):
                    if ag not in age:
                        i += 1
                        continue
                elif ag != age:
                    i += 1
                    continue
    
            # Filter by handiness
            if handiness is not None and hand != handiness:
                i += 1
                continue
    
            results_arr.append([power, phase, s_id, t_id, gend, hand, ag, label, img, task_type])
            i += 1
    
        return results_arr

In [3]:
def _aggregate_by_subject(results):
    # subj_band_vals[(ch, band)][sid] -> [vals_per_trial...]
    subj_band_vals = defaultdict(lambda: defaultdict(list))
    for power, phase, s_id, t_id, gender, handiness, age, label, img, task_type in results:
        s_id = int(s_id)
        vecs = (power.mean(axis=2) if power.ndim == 3 else power).astype(np.float32, copy=False)  # (C,F)
        for ch in range(n_channels):
            for band, idx in band_cols.items():
                if idx.size == 0: 
                    continue
                val_trial = float(np.nanmean(vecs[ch, idx]))
                subj_band_vals[(ch, band)][s_id].append(val_trial)
    # среднее по триалам → массивы
    subj_vecs = {}
    for ch in range(n_channels):
        for band in bands.keys():
            ids, vals = [], []
            d = subj_band_vals[(ch, band)]
            for sid, arr in d.items():
                if arr:
                    ids.append(sid)
                    vals.append(float(np.nanmean(arr)))
            subj_vecs[(ch, band)] = {'ids': ids, 'vals': np.array(vals, float)}
    return subj_vecs

def build_subject_vectors(
    day_subj: Dict[Tuple[int, str], Dict[str, Any]],
    night_subj: Dict[Tuple[int, str], Dict[str, Any]],
    bands: Dict[str, Tuple[float, float]],
    n_channels: int,
) -> Dict[Tuple[int, str], Dict[str, Any]]:
    """
    Собирает subject_vectors для каждого (канал, бэнд) без парного выравнивания.

    Parameters
    ----------
    day_subj, night_subj : dict
        {(ch, band): {'ids': List[int], 'vals': np.ndarray}, ...}
        — результат aggregate_by_subject(...)
    bands : dict
        Словарь бэндов; используются только ключи (имена бэндов).
    n_channels : int
        Количество каналов.

    Returns
    -------
    subject_vectors : dict
        {(ch, band): {
            'day': np.ndarray, 'night': np.ndarray,
            'day_ids': List[int], 'night_ids': List[int]
        }}
    """
    subject_vectors: Dict[Tuple[int, str], Dict[str, Any]] = {}

    for ch in range(n_channels):
        for band in bands.keys():
            key = (ch, band)
            d = day_subj.get(key, {'ids': [], 'vals': np.array([], dtype=float)})
            n = night_subj.get(key, {'ids': [], 'vals': np.array([], dtype=float)})

            d_ids: List[int] = list(d.get('ids', []))
            n_ids: List[int] = list(n.get('ids', []))
            d_vals = np.asarray(d.get('vals', np.array([], float)), dtype=float)
            n_vals = np.asarray(n.get('vals', np.array([], float)), dtype=float)

            subject_vectors[key] = {
                'day'      : d_vals,
                'night'    : n_vals,
                'day_ids'  : d_ids,
                'night_ids': n_ids,
            }

    return subject_vectors

In [51]:
def hedges_g(x, y) -> float:
    """
    Hedges' g для двух независимых выборок (Day − Night), с поправкой J.
    """
    x = np.asarray(x, float); y = np.asarray(y, float)
    nx, ny = len(x), len(y)
    if nx < 2 or ny < 2:
        return np.nan
    vx, vy = np.var(x, ddof=1), np.var(y, ddof=1)
    # обе дисперсии нулевые
    if vx == 0 and vy == 0:
        return float(np.sign(np.nanmean(x) - np.nanmean(y)) * 0.0)
    # объединённая (пул) СКО
    sp2 = ((nx - 1) * vx + (ny - 1) * vy) / (nx + ny - 2)
    if sp2 <= 0:
        return np.nan
    d = (np.nanmean(x) - np.nanmean(y)) / np.sqrt(sp2)  # Day − Night
    J = 1 - 3 / (4 * (nx + ny) - 9) if (nx + ny) > 2 else 1.0
    return float(J * d)


def build_band_table(subject_vectors: Dict[Tuple[int, str], Dict[str, Any]],
                     alpha: float = 0.05) -> pd.DataFrame:
    """
    Строит сводную таблицу по всем (канал, бэнд) из subject_vectors:
      Welch t-test, p-value, Hedges' g, примерная мощность (TTestIndPower).
    """
    power_calc = TTestIndPower()
    rows = []

    for (ch, band), vecs in subject_vectors.items():
        x = np.asarray(vecs.get('day',   []), float)   # day
        y = np.asarray(vecs.get('night', []), float)   # night
        n1, n2 = len(x), len(y)
        if n1 == 0 or n2 == 0:
            continue

        # Welch t-test (устойчивей при разн. дисперсиях)
        try:
            t_stat, p_val = stats.ttest_ind(x, y, equal_var=False, nan_policy='omit')
        except Exception:
            t_stat, p_val = np.nan, np.nan

        # Эффект (Hedges' g), знак Day − Night
        g = hedges_g(x, y)

        # Прикидка мощности (допущение равных дисперсий в модели мощности)
        try:
            ratio = n2 / max(n1, 1)
            es = abs(g)
            power = power_calc.solve_power(effect_size=es, nobs1=n1, ratio=ratio,
                                           alpha=alpha, alternative='two-sided')
        except Exception:
            power = np.nan

        rows.append({
            'channel': ch,
            'band': band,
            'n_day': n1,
            'n_night': n2,
            'mean_day': float(np.nanmean(x)) if n1 else np.nan,
            'mean_night': float(np.nanmean(y)) if n2 else np.nan,
            'delta_day_minus_night': float(np.nanmean(x) - np.nanmean(y)) if (n1 and n2) else np.nan,
            't_stat': float(t_stat) if np.isfinite(t_stat) else np.nan,
            'p_value': float(p_val) if np.isfinite(p_val) else np.nan,
            'hedges_g': float(g) if np.isfinite(g) else np.nan,
            'power': float(power) if np.isfinite(power) else np.nan,
            'sig_alpha_0.05': bool((p_val <= 0.05) if np.isfinite(p_val) else False),
            'sig_alpha_0.01': bool((p_val <= 0.01) if np.isfinite(p_val) else False),
            'sig_and_power': bool((p_val <= 0.05) and (power >= 0.8)) if (np.isfinite(p_val) and np.isfinite(power)) else False,
        })

    df = pd.DataFrame(rows)
    if not df.empty:
        df = df.sort_values(['band', 'p_value', 'power'],
                            ascending=[True, True, False]).reset_index(drop=True)
    return df

# --- пример использования ---
# band_table = build_band_table(subject_vectors, alpha=0.05)
# print(band_table.head(12))




In [38]:
# === 1) Грузим сразу готовые поднаборы: Day и Evening (будем называть Night) ===
exec_spec_path     = r'./Generated/Spectrums/exec_and_rest_morlets.npz'
day_time_meta_path = r"./Supplementary/Experiment_Metadata.xlsx"

results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="g")
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="g")  # Evening ~ Night

print(f"Day trials: {len(results_day)}, Night trials: {len(results_night)}")

# Размерности из первого доступного набора
first_power = (results_day or results_night)[0][0]
n_channels, n_freqs = first_power.shape[:2]

# Частоты (если нет внешнего freqs/psd_freqs)
if 'freqs' in globals() and len(freqs) == n_freqs:
    f = np.asarray(freqs)
elif 'psd_freqs' in globals() and len(psd_freqs) == n_freqs:
    f = np.asarray(psd_freqs)
else:
    f = np.linspace(0.0, 60.0, n_freqs, endpoint=False)  # fallback

bands = {'Delta':(1,4), 'Tetta':(4,7), 'Alpha':(7,13), 'Beta':(13,30)}
band_cols = {name: np.where((f >= lo) & (f < hi))[0] for name,(lo,hi) in bands.items()}


Day trials: 238, Night trials: 196


In [45]:

# (b) Вектора по субъектам (НЕ парные): усредняем по триалам субъекта в каждом бэнде


day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

# Итоговая структура как раньше (раздельные day/night, без выравнивания пар)
subject_vectors = {}
for ch in range(n_channels):
    for band in bands.keys():
        subject_vectors[(ch, band)] = {
            'day'      : day_subj[(ch, band)]['vals'],
            'night'    : night_subj[(ch, band)]['vals'],
            'day_ids'  : day_subj[(ch, band)]['ids'],
            'night_ids': night_subj[(ch, band)]['ids'],
        }

# Пример проверки
#k = (0, 'Alpha')
#print("Alpha, ch0: day_n=", subject_vectors[k]['day'].size, " night_n=", subject_vectors[k]['night'].size)


In [46]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.power import TTestIndPower

alpha = 0.05
power_calc = TTestIndPower()

def hedges_g(x, y):
    x = np.asarray(x, float); y = np.asarray(y, float)
    nx, ny = len(x), len(y)
    if nx < 2 or ny < 2:
        return np.nan
    vx, vy = np.var(x, ddof=1), np.var(y, ddof=1)
    if vx == 0 and vy == 0:
        return np.sign(np.mean(x)-np.mean(y)) * 0.0
    sp = np.sqrt(((nx-1)*vx + (ny-1)*vy) / (nx + ny - 2))
    if sp == 0:
        return np.nan
    d = (np.mean(x) - np.mean(y)) / sp                   # Day − Night
    J = 1 - 3/(4*(nx + ny) - 9) if (nx+ny) > 2 else 1.0  # small-sample correction
    return J * d

rows = []
for (ch, band), vecs in subject_vectors.items():
    x = np.asarray(vecs['day'], float)    # длина ≈ 9
    y = np.asarray(vecs['night'], float)  # длина ≈ 7

    n1, n2 = len(x), len(y)
    if n1 == 0 or n2 == 0:
        continue

    # Welch's t-test
    try:
        t_stat, p_val = stats.ttest_ind(x, y, equal_var=False, nan_policy='omit')
    except Exception:
        t_stat, p_val = np.nan, np.nan

    # Effect size (Hedges' g) — знак = Day − Night
    g = hedges_g(x, y)

    # Power (приближение равных дисперсий в power-анализе — допустимо как грубая оценка)
    try:
        ratio = n2 / max(n1, 1)
        power = power_calc.solve_power(effect_size=abs(g), nobs1=n1, ratio=ratio,
                                       alpha=alpha, alternative='two-sided')
    except Exception:
        power = np.nan

    rows.append({
        'channel': ch,
        'band': band,
        'n_day': n1,
        'n_night': n2,
        'mean_day': float(np.nanmean(x)) if n1 else np.nan,
        'mean_night': float(np.nanmean(y)) if n2 else np.nan,
        'delta_day_minus_night': float(np.nanmean(x) - np.nanmean(y)) if (n1 and n2) else np.nan,
        't_stat': float(t_stat) if np.isfinite(t_stat) else np.nan,
        'p_value': float(p_val) if np.isfinite(p_val) else np.nan,
        'hedges_g': float(g) if np.isfinite(g) else np.nan,
        'power': float(power) if np.isfinite(power) else np.nan,
        'sig_alpha_0.05': bool((p_val <= 0.05) if np.isfinite(p_val) else False),
        'sig_alpha_0.01': bool((p_val <= 0.01) if np.isfinite(p_val) else False),
        'sig_and_power': bool((p_val <= 0.05) and (power >= 0.8)) if (np.isfinite(p_val) and np.isfinite(power)) else False,
    })

band_table = pd.DataFrame(rows).sort_values(['band', 'p_value', 'power'], ascending=[True, True, False]).reset_index(drop=True)
print(band_table.head(12))


    channel   band  n_day  n_night  mean_day  mean_night  \
0         1  Alpha      9        7  0.206129    0.131978   
1        32  Alpha      9        7  0.145488    0.101429   
2        29  Alpha      9        7  0.210172    0.129525   
3        33  Alpha      9        7  0.158877    0.110073   
4        16  Alpha      9        7  0.292725    0.163951   
5        60  Alpha      9        7  0.157491    0.111808   
6        43  Alpha      9        7  0.195319    0.125849   
7        10  Alpha      9        7  0.158518    0.096008   
8        53  Alpha      9        7  0.169641    0.115450   
9        61  Alpha      9        7  0.176813    0.125495   
10       40  Alpha      9        7  0.162569    0.124282   
11       28  Alpha      9        7  0.159240    0.117491   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.074152  1.219559  0.248999  0.525112  0.163211   
1                0.044059  1.170721  0.263722  0.515758  0.159111   
2           

In [71]:
results_day_r   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="r")
results_night_r = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="r")

day_subj   = _aggregate_by_subject(results_day_r)
night_subj = _aggregate_by_subject(results_night_r)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))


    channel   band  n_day  n_night  mean_day  mean_night  \
0        28  Alpha      9        7  0.167016    0.121071   
1        21  Alpha      9        7  0.158835    0.106328   
2        39  Alpha      9        7  0.115203    0.081778   
3         1  Alpha      9        7  0.201233    0.140555   
4        60  Alpha      9        7  0.162024    0.120873   
5        43  Alpha      9        7  0.186887    0.137480   
6        10  Alpha      9        7  0.143358    0.103166   
7        29  Alpha      9        7  0.213793    0.144271   
8        30  Alpha      9        7  0.210003    0.477281   
9        14  Alpha      9        7  0.191316    0.148728   
10       26  Alpha      9        7  0.138376    0.098535   
11       16  Alpha      9        7  0.240926    0.172717   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.045945  1.015423  0.329128  0.448145  0.131779   
1                0.052507  0.984423  0.343059  0.436845  0.127607   
2           

In [72]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="g")
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="r")

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))


    channel   band  n_day  n_night  mean_day  mean_night  \
0         1  Alpha      9        7  0.206129    0.140555   
1        16  Alpha      9        7  0.292725    0.172717   
2        10  Alpha      9        7  0.158518    0.103166   
3        29  Alpha      9        7  0.210172    0.144271   
4        43  Alpha      9        7  0.195319    0.137480   
5        30  Alpha      9        7  0.202103    0.477281   
6        28  Alpha      9        7  0.159240    0.121071   
7        40  Alpha      9        7  0.162569    0.128939   
8        60  Alpha      9        7  0.157491    0.120873   
9        39  Alpha      9        7  0.112730    0.081778   
10       21  Alpha      9        7  0.160653    0.106328   
11        9  Alpha      9        7  0.204354    0.391305   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.065575  1.058917  0.311962  0.459074  0.135922   
1                0.120008  1.051773  0.317298  0.450258  0.132572   
2           

In [73]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="r")
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="g")

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        33  Alpha      9        7  0.162259    0.110073   
1        32  Alpha      9        7  0.145046    0.101429   
2        43  Alpha      9        7  0.186887    0.125849   
3        60  Alpha      9        7  0.162024    0.111808   
4        53  Alpha      9        7  0.175105    0.115450   
5        35  Alpha      9        7  0.158489    0.110467   
6        21  Alpha      9        7  0.158835    0.097482   
7        14  Alpha      9        7  0.191316    0.135142   
8        28  Alpha      9        7  0.167016    0.117491   
9        10  Alpha      9        7  0.143358    0.096008   
10       29  Alpha      9        7  0.213793    0.129525   
11       61  Alpha      9        7  0.182807    0.125495   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.052186  1.249587  0.235300  0.547339  0.173260   
1                0.043617  1.211743  0.247580  0.536753  0.168420   
2           

In [74]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, gender='m')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, gender='m')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        39  Alpha      3        5  0.094528    0.066300   
1        54  Alpha      3        5  0.123549    0.076123   
2        32  Alpha      3        5  0.133290    0.083263   
3        43  Alpha      3        5  0.173199    0.111513   
4         1  Alpha      3        5  0.201483    0.112271   
5        10  Alpha      3        5  0.122833    0.083920   
6        35  Alpha      3        5  0.121016    0.093009   
7         7  Alpha      3        5  0.088818    0.064735   
8        60  Alpha      3        5  0.144235    0.092066   
9        13  Alpha      3        5  0.149932    0.097910   
10       33  Alpha      3        5  0.141721    0.099590   
11       21  Alpha      3        5  0.136749    0.075242   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.028228  2.046505  0.096571  1.017244  0.217969   
1                0.047425  2.128560  0.105813  1.423601  0.375081   
2           

In [75]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, gender='f')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, gender='f')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        51  Alpha      6        2  0.142778    0.245317   
1        12  Alpha      6        2  0.177184    0.336282   
2         0  Alpha      6        2  0.165706    0.233322   
3        30  Alpha      6        2  0.186921    0.325197   
4        50  Alpha      6        2  0.181766    0.299571   
5        11  Alpha      6        2  0.131324    0.205926   
6        42  Alpha      6        2  0.182887    0.307004   
7        20  Alpha      6        2  0.209187    0.172119   
8        62  Alpha      6        2  0.243797    0.190975   
9        44  Alpha      6        2  0.233628    0.188211   
10       17  Alpha      6        2  0.323619    0.239523   
11       22  Alpha      6        2  0.140391    0.196354   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0               -0.102539 -0.837235  0.519413 -0.692044  0.111115   
1               -0.159098 -0.811027  0.533901 -0.694791  0.111610   
2           

In [76]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="r", gender='m')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="r", gender='m')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        39  Alpha      3        5  0.097750    0.069560   
1        32  Alpha      3        5  0.142665    0.089594   
2        60  Alpha      3        5  0.154588    0.097650   
3        35  Alpha      3        5  0.137778    0.099079   
4        43  Alpha      3        5  0.185732    0.119652   
5        13  Alpha      3        5  0.160143    0.108820   
6        54  Alpha      3        5  0.132819    0.084781   
7        28  Alpha      3        5  0.142541    0.104791   
8        16  Alpha      3        5  0.233433    0.159399   
9         1  Alpha      3        5  0.196747    0.117464   
10       11  Alpha      3        5  0.152269    0.081324   
11       10  Alpha      3        5  0.126134    0.089918   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.028190  1.852148  0.131641  0.896316  0.179965   
1                0.053072  1.718915  0.139517  0.989767  0.208929   
2           

In [77]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="g", gender='m')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="g", gender='m')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        39  Alpha      3        5  0.092918    0.064670   
1        54  Alpha      3        5  0.118913    0.071794   
2        43  Alpha      3        5  0.166933    0.107444   
3        32  Alpha      3        5  0.128602    0.080098   
4         1  Alpha      3        5  0.203851    0.109674   
5        33  Alpha      3        5  0.136631    0.090939   
6        10  Alpha      3        5  0.121182    0.080921   
7         7  Alpha      3        5  0.087250    0.062828   
8        35  Alpha      3        5  0.112635    0.089974   
9        60  Alpha      3        5  0.139059    0.089274   
10       13  Alpha      3        5  0.144826    0.092455   
11       21  Alpha      3        5  0.132361    0.070246   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.028248  2.134753  0.081846  1.085617  0.241453   
1                0.047119  2.253681  0.106146  1.600746  0.452967   
2           

In [78]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="r", gender='m')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="g", gender='m')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        39  Alpha      3        5  0.097750    0.064670   
1        35  Alpha      3        5  0.137778    0.089974   
2        33  Alpha      3        5  0.151901    0.090939   
3        32  Alpha      3        5  0.142665    0.080098   
4        13  Alpha      3        5  0.160143    0.092455   
5         7  Alpha      3        5  0.091952    0.062828   
6        60  Alpha      3        5  0.154588    0.089274   
7        57  Alpha      3        5  0.136253    0.096606   
8        10  Alpha      3        5  0.126134    0.080921   
9        61  Alpha      3        5  0.156417    0.104151   
10       43  Alpha      3        5  0.185732    0.107444   
11       54  Alpha      3        5  0.132819    0.071794   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.033080  2.659229  0.049432  1.297374  0.322235   
1                0.047804  2.457639  0.054341  1.452029  0.387344   
2           

In [79]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, gender='m')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, gender='f')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        34  Alpha      3        2  0.138703    0.191623   
1         3  Alpha      3        2  0.157664    0.217659   
2        58  Alpha      3        2  0.114828    0.185502   
3        48  Alpha      3        2  0.179681    0.320212   
4        12  Alpha      3        2  0.141394    0.336282   
5        50  Alpha      3        2  0.141706    0.299571   
6        47  Alpha      3        2  0.165674    0.320897   
7        27  Alpha      3        2  0.103070    0.163158   
8        32  Alpha      3        2  0.133290    0.162986   
9        38  Alpha      3        2  0.080419    0.130507   
10       18  Alpha      3        2  0.134207    0.258900   
11       46  Alpha      3        2  0.134858    0.259428   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0               -0.052920 -1.812948  0.239806 -1.334030  0.180245   
1               -0.059995 -1.548959  0.268992 -1.088497  0.137047   
2           

In [80]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="g", gender='m')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="r", gender='m')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0         1  Alpha      3        5  0.203851    0.117464   
1        39  Alpha      3        5  0.092918    0.069560   
2        32  Alpha      3        5  0.128602    0.089594   
3        43  Alpha      3        5  0.166933    0.119652   
4        54  Alpha      3        5  0.118913    0.084781   
5        10  Alpha      3        5  0.121182    0.089918   
6        60  Alpha      3        5  0.139059    0.097650   
7         7  Alpha      3        5  0.087250    0.068550   
8        51  Alpha      3        5  0.209290    0.104389   
9        21  Alpha      3        5  0.132361    0.085233   
10       11  Alpha      3        5  0.130423    0.081324   
11       13  Alpha      3        5  0.144826    0.108820   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.086387  1.690137  0.169794  1.119970  0.253765   
1                0.023358  1.471394  0.201206  0.732771  0.136204   
2           

In [81]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, gender='f')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, gender='m')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        60  Alpha      6        5  0.166385    0.092066   
1        32  Alpha      6        5  0.151366    0.083263   
2        35  Alpha      6        5  0.169201    0.093009   
3        28  Alpha      6        5  0.178655    0.102462   
4        29  Alpha      6        5  0.234269    0.110821   
5        61  Alpha      6        5  0.195967    0.112272   
6        21  Alpha      6        5  0.171697    0.075242   
7        33  Alpha      6        5  0.169146    0.099590   
8        39  Alpha      6        5  0.123067    0.066300   
9        10  Alpha      6        5  0.168780    0.083920   
10       26  Alpha      6        5  0.152240    0.081220   
11       23  Alpha      6        5  0.161179    0.083291   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.074319  1.309770  0.231807  0.676033  0.170225   
1                0.068103  1.310827  0.234666  0.671347  0.168526   
2           

In [82]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="g", gender='f')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="g", gender='f')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        30  Alpha      6        2  0.186275    0.336738   
1        51  Alpha      6        2  0.146187    0.226325   
2        12  Alpha      6        2  0.177934    0.295563   
3         0  Alpha      6        2  0.168796    0.222825   
4        62  Alpha      6        2  0.239790    0.183245   
5        20  Alpha      6        2  0.202665    0.164414   
6        50  Alpha      6        2  0.177968    0.268112   
7         9  Alpha      6        2  0.206587    0.183830   
8        11  Alpha      6        2  0.135112    0.193508   
9        44  Alpha      6        2  0.230227    0.185035   
10       42  Alpha      6        2  0.187694    0.277327   
11       10  Alpha      6        2  0.177186    0.133725   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0               -0.150463 -0.814434  0.549457 -0.870665  0.147595   
1               -0.080138 -0.737430  0.550325 -0.537202  0.086532   
2           

In [83]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="r", gender='f')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="r", gender='f')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0         0  Alpha      6        2  0.159527    0.254316   
1        51  Alpha      6        2  0.135960    0.283300   
2        12  Alpha      6        2  0.175685    0.417721   
3        30  Alpha      6        2  0.188212    0.302113   
4        50  Alpha      6        2  0.189363    0.362488   
5        11  Alpha      6        2  0.123748    0.230762   
6        42  Alpha      6        2  0.173273    0.366358   
7        22  Alpha      6        2  0.126843    0.226060   
8        47  Alpha      6        2  0.251838    0.387057   
9        32  Alpha      6        2  0.146237    0.179444   
10       17  Alpha      6        2  0.350875    0.244128   
11       18  Alpha      6        2  0.196852    0.292957   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0               -0.094789 -0.949657  0.469844 -0.751532  0.122296   
1               -0.147340 -0.969181  0.487944 -0.984026  0.175238   
2           

In [84]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="g", gender='f')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="r", gender='f')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        30  Alpha      6        2  0.186275    0.302113   
1        12  Alpha      6        2  0.177934    0.417721   
2         0  Alpha      6        2  0.168796    0.254316   
3        51  Alpha      6        2  0.146187    0.283300   
4        50  Alpha      6        2  0.177968    0.362488   
5        47  Alpha      6        2  0.235080    0.387057   
6        42  Alpha      6        2  0.187694    0.366358   
7        11  Alpha      6        2  0.135112    0.230762   
8        58  Alpha      6        2  0.161207    0.198592   
9        46  Alpha      6        2  0.189524    0.291404   
10       22  Alpha      6        2  0.147165    0.226060   
11       45  Alpha      6        2  0.207055    0.323816   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0               -0.115838 -0.903084  0.497452 -0.779712  0.127930   
1               -0.239787 -0.924202  0.503356 -0.931226  0.161935   
2           

In [85]:
results_day   = filter_exec_psds(exec_spec_path, day_time="Day",     day_time_meta_path=day_time_meta_path, stim_type="r", gender='f')
results_night = filter_exec_psds(exec_spec_path, day_time="Evening", day_time_meta_path=day_time_meta_path, stim_type="g", gender='f')

day_subj   = _aggregate_by_subject(results_day)
night_subj = _aggregate_by_subject(results_night)

subject_vectors = build_subject_vectors(day_subj, night_subj, bands, n_channels)

band_table_r = build_band_table(subject_vectors, alpha=0.05)
print(band_table_r.head(12))

    channel   band  n_day  n_night  mean_day  mean_night  \
0        20  Alpha      6        2  0.222231    0.164414   
1        51  Alpha      6        2  0.135960    0.226325   
2         0  Alpha      6        2  0.159527    0.222825   
3        30  Alpha      6        2  0.188212    0.336738   
4        12  Alpha      6        2  0.175685    0.295563   
5        62  Alpha      6        2  0.251812    0.183245   
6        11  Alpha      6        2  0.123748    0.193508   
7        44  Alpha      6        2  0.240432    0.185035   
8        15  Alpha      6        2  0.259614    0.190221   
9        42  Alpha      6        2  0.173273    0.277327   
10       17  Alpha      6        2  0.350875    0.237221   
11       22  Alpha      6        2  0.126843    0.181501   

    delta_day_minus_night    t_stat   p_value  hedges_g     power  \
0                0.057818  0.758565  0.479417  0.302518  0.061467   
1               -0.090365 -0.860457  0.506767 -0.691176  0.110959   
2           